In [10]:
# 라이브러리 

import numpy as np
import pandas as pd
import lightgbm as lgb
from tqdm import tqdm
from collections import Counter
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler

import warnings
warnings.filterwarnings('ignore')
np.random.seed(42)

In [21]:
# 데이터 불러오기

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train = train.drop(['ID_code'],1)
sub = pd.DataFrame(test['ID_code'])   #추후 제출용을 위해
test = test.drop(['ID_code'],1)

# train data의 feature 변수와 target 변수 분리
X = train.drop(['target'], axis=1)
y = train['target']

In [12]:
## (Modeling file과 동일한 과정)

# 클래스 불균형을 해소하기 위한 oversampling

oversample = RandomOverSampler(sampling_strategy=0.33,random_state= 42)
X_over, y_over = oversample.fit_resample(X, y)

# 샘플링 후 클래스 0과 클래스 1의 갯수 확인 

print(Counter(y_over))

# RandomSearchCV()를 통해 최적 파라미터 설정 후 모델 생성

clf = lgb.LGBMClassifier(bagging_fraction=0.85, bagging_freq=1, colsample_bytree=1,
               feature_fraction=1, importance_type='split',
               learning_rate=0.1, max_bin=256, max_depth=1,
               min_child_samples=153, min_child_weight=0.1,
               n_estimators=4000, n_jobs=-1, num_leaves=4, num_threads=8,
               objective='binary', reg_alpha=0.1,
               reg_lambda=0, seed=500, silent=True, subsample=0.8)

Counter({0: 143922, 1: 47494})


In [13]:
# 모델 학습

clf.fit(X_over, y_over)

[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_fraction is set=0.85, subsample=0.8 will be ignored. Current value: bagging_fraction=0.85
[LightGBM] [Warning] num_threads is set=8, n_jobs=-1 will be ignored. Current value: num_threads=8
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


LGBMClassifier(bagging_fraction=0.85, bagging_freq=1, colsample_bytree=1,
               feature_fraction=1, max_bin=256, max_depth=1,
               min_child_samples=153, min_child_weight=0.1, n_estimators=4000,
               num_leaves=4, num_threads=8, objective='binary', reg_alpha=0.1,
               reg_lambda=0, seed=500, silent=True, subsample=0.8)

In [14]:
# 모델 적합

pred = clf.predict(test)

In [25]:
# 데이터 csv로 추출

sub['target'] = pd.DataFrame(pred)
sub.columns=['ID_code','target']
sub.to_csv('Submission_Team3.csv',index = False)